## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

import torch
import torch.nn as nn
import pandas as pd

from collections import OrderedDict 
from sklearn import metrics, model_selection
from torch.optim import Adam

sys.path.append(os.path.abspath(''))

import utils.more_torch_functions as mtf
import datasets

from utils.custom_activations import StepActivation, StepFunction
from utils.modules import Parallel, MaxLayer, MaxHierarchicalLayer
from utils.misc import cross_valid, combine_prompts, cov_score

# torch.autograd.set_detect_anomaly(True)

## Load data

In [2]:
np_x, np_y = datasets.DiabetesDataset.get_dataset(balancing=True, discretizing=False, hot_encoding=True)
x_data, y_data = torch.Tensor(np_x), torch.Tensor(np_y)
input_size = x_data.size(1)
print(x_data.size())

torch.Size([536, 8])


## Hooks

In [3]:
intermediate_outputs = {}
def get_intermediate_outputs(name):
    def hook(model, input, output):
        if model.training:
            intermediate_outputs.setdefault(name, dict())["train"] = output
        else:
            intermediate_outputs.setdefault(name, dict())["valid"] = output
    return hook

def true_label_for_backward(train, valid):
    def hook(model, input):
        if model.training:
            model.true_labels = train
        else:
            model.true_labels = valid
    return hook

# créer hook fonction de perte pour meilleur backward ? (comparer individuellement les sorties des réseaux ???)

## Networks

### Network parts

In [4]:
class ApproxNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        hl1 = 10

        self.nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', StepActivation()),
            ('l2', nn.Linear(hl1,1)),
            ('a2', StepActivation())
        ]))        

    def forward(self, x):
        x = self.nn(x)

        return x

class BigNet(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self. nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', nn.Sigmoid()),
            ('l2', nn.Linear(hl1,hl2)),
            ('a2', nn.Sigmoid()),
            ('l3', nn.Linear(hl2,1)),
            ('a3', StepActivation()),
        ]))
    
    def forward(self, x):
        x = self.nn(x)

        return x

### Previous Network (and related)

In [5]:
class NetResults():
    def __init__(self, *tensors):
        for tensor in tensors:
            self.register_result(tensor)

    def __getattr__(self, name):
        if hasattr(self.x, name):
            return getattr(self.x, name)
        else:
            raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")
    
    def __dir__(self):
        return dir(self.x)

    def __str__(self):
        return '\n'.join([str(t) for t in self.tensors()])

    def tensors(self):
        for v in self.__dict__.values():
            yield v

    def detach(self):
        for t in self.tensors():
            t.detach()
        return self
    
    def round(self, *args):
        for t in self.tensors():
            t.round(*args)
        return self

class Netv1(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.a1 = ApproxNet()
        self.a2 = ApproxNet()
        self.nn = BigNet()

    def forward(self, x):
        xa1 = self.a1(x)
        xa2 = self.a2(x)
        xnn = self.nn(x)

        res = [xnn, xa1, xa2]

        # /!\ to change for backward propagation /!\
        x = mtf.bitwise_big_or(*[(torch.round(t)).to(bool) for t in res])
        # maximum ???
        # xmax = mtf.maximum(res)
        # x = torch.where(xmax > 0.5, xmax, xnn)

        x = NetResults(x, *res)

        return x

### New Network definition

In [6]:
class Netv2(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(OrderedDict([
            ('nets', Parallel(OrderedDict([
                ('nn', BigNet()),
                ('apx1', ApproxNet()),
                ('apx2', ApproxNet()),
                ('apx3', ApproxNet()),
            ]))),
            ('or_', MaxLayer()),
        ]))

    def forward(self, input):
        return self.net(input)

## Network evaluation

In [7]:
model = Netv2()
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=1e-2, weight_decay=1e-6)

model.net.nets.register_forward_hook(get_intermediate_outputs("parallel_out"))
hook_label = lambda y_train, y_test : model.net.or_.register_forward_pre_hook(true_label_for_backward(y_train, y_test))

skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=104)
bnet_split_res = cross_valid(x_data, y_data, model, criterion, optimizer, skf, hook_label, max_epoch=500)

nn_children = [name for name, _ in model.net.nets.named_children()]
dict_metrics = {(modelname, metric, key): list() for modelname in ["net"] + nn_children
                for metric in ("f1score", "coverage0", "coverage1") for key in ("valid", "train")}

for i, (train_pred, train_true, valid_pred, valid_true) in enumerate(bnet_split_res):
    out_nns = intermediate_outputs["parallel_out"]
    for d in out_nns.values():
        for k, v in d.items():
            d[k] = v.detach().round()

    f1prompts = []
    covprompts = []
    sep_model = f"{'|':^9}"
    for k, pred, true in [["valid", valid_pred, valid_true], ["train", train_pred, train_true]]:
        net_f1_score = metrics.f1_score(true, pred, average="binary")
        net_cov_score = cov_score(true, pred)
        dict_metrics[('net', 'f1score', k)].append(net_f1_score)
        dict_metrics[('net', 'coverage1', k)].append(net_cov_score[1])
        dict_metrics[('net', 'coverage0', k)].append(net_cov_score[0])
        covprompts.append(f"{'Net':<15}{net_cov_score[0]:.3f}{sep_model}{'Net':<15}{net_cov_score[1]:.3f}")

        prev_modelname = 'Net'
        prev_f1_score = net_f1_score

        for c, cname in enumerate(nn_children):
            modelname = 'Big Network'if c==0 else f'Approx {c}'
            model_pred = out_nns[k][cname]

            model_f1_score = metrics.f1_score(true, model_pred, average="binary")
            model_cov_score = cov_score(true, model_pred)

            if c%2==0:
                f1prompts.append(f"{prev_modelname:<15}{prev_f1_score:.3f}{sep_model}{modelname:<15}{model_f1_score:.3f}")
            else:
                prev_modelname = modelname
                prev_f1_score = model_f1_score
            covprompts.append(f"{modelname:<15}{model_cov_score[0]:.3f}{sep_model}{modelname:<15}{model_cov_score[1]:.3f}")

            dict_metrics[(cname, 'f1score', k)].append(model_f1_score)
            dict_metrics[(cname, 'coverage1', k)].append(model_cov_score[1])
            dict_metrics[(cname, 'coverage0', k)].append(model_cov_score[0])

        if c%2:
            f1prompts.append(f"{modelname:<15}{model_f1_score:.3f}{sep_model}{'':<20}")

    sep_tv = f"{'||':^10}"
    print(f"Fold {i+1:3} :            {'Valid':^49}{sep_tv}{'Train':^49}",
        f"\tF1 score      {combine_prompts(f1prompts, sep_tv)}",
        f"\tCoverage      {combine_prompts(covprompts, sep_tv)}",
        sep='\n')

Fold   1 :                                  Valid                          ||                          Train                      
	F1 score      Net            0.746    |    CentralNet     0.704    ||    Net            0.941    |    CentralNet     0.966
	              Approx 1       0.537    |    Approx 2       0.455    ||    Approx 1       0.412    |    Approx 2       0.561
	              Approx 3       0.619    |                            ||    Approx 3       0.598    |                        
	Coverage      Net            0.630    |    Net            0.815    ||    Net            0.917    |    Net            0.963
	              CentralNet     0.704    |    CentralNet     0.704    ||    CentralNet     1.000    |    CentralNet     0.934
	              Approx 1       0.889    |    Approx 1       0.407    ||    Approx 1       0.992    |    Approx 1       0.261
	              Approx 2       0.741    |    Approx 2       0.370    ||    Approx 2       0.967    |    Approx 2       0.402
	

In [8]:
df_metrics = pd.DataFrame.from_dict(dict_metrics, orient='index')
mean_metrics = df_metrics.mean(axis=1)

f1_mean_prompts = []
cov_mean_prompts = []

for k in ["valid", "train"]:
    prev_modelname = 'Net'
    prev_f1_avg = mean_metrics[('net', 'f1score', 'valid')]
    cov_mean_prompts.append(f"{'Net':<15}{mean_metrics[('net', 'coverage0', k)]:.3f}{sep_model}{'Net':<15}{mean_metrics[('net', 'coverage1', k)]:.3f}")
    for c, cname in enumerate(nn_children):
        modelname = 'CentralNet'if c==0 else f'Approx {c}'
        model_f1_avg = mean_metrics[(cname, 'f1score', k)]
        model_cov0_avg = mean_metrics[(cname, 'coverage0', k)]
        model_cov1_avg = mean_metrics[(cname, 'coverage1', k)]

        if c%2==0:
            f1_mean_prompts.append(f"{prev_modelname:<15}{prev_f1_avg:.3f}{sep_model}{modelname:<15}{model_f1_avg:.3f}")
        else:
            prev_modelname = modelname
            prev_f1_score = model_f1_score

        cov_mean_prompts.append(f"{modelname:<15}{model_cov0_avg:.3f}{sep_model}{modelname:<15}{model_cov1_avg:.3f}")
    
    if c%2:
        f1_mean_prompts.append(f"{modelname:<15}{model_f1_score:.3f}{sep_model}{'':<20}")

print(f"Average  :            {'Valid':^49}{sep_tv}{'Train':^49}",
      f"\tF1 score      {combine_prompts(f1_mean_prompts, sep_tv)}",
      f"\tCoverage      {combine_prompts(cov_mean_prompts, sep_tv)}",
      sep='\n')

Average  :                                  Valid                          ||                          Train                      
	F1 score      Net            0.786    |    CentralNet     0.525    ||    Net            0.786    |    CentralNet     0.680
	              Approx 1       0.786    |    Approx 2       0.520    ||    Approx 1       0.786    |    Approx 2       0.610
	              Approx 3       0.573    |                            ||    Approx 3       0.573    |                        
	Coverage      Net            0.705    |    Net            0.836    ||    Net            0.900    |    Net            0.943
	              CentralNet     0.873    |    CentralNet     0.494    ||    CentralNet     1.000    |    CentralNet     0.661
	              Approx 1       0.884    |    Approx 1       0.448    ||    Approx 1       0.964    |    Approx 1       0.492
	              Approx 2       0.877    |    Approx 2       0.400    ||    Approx 2       0.968    |    Approx 2       0.461
	

In [9]:
model_ref = BigNet()
criterion_ref = nn.BCELoss()
optimizer_ref = Adam(model_ref.parameters(), lr=1e-2, weight_decay=1e-6)
skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=104)
ref_split_res = cross_valid(x_data, y_data, model_ref, criterion_ref, optimizer_ref, skf, max_epoch=500)

ref_metrics = {('ref_model', metric, key): list() for metric in ("f1score", "coverage0", "coverage1") for key in ("valid", "train")}
for train_pred, train_true, valid_pred, valid_true in ref_split_res:
    for k, pred, true in [["valid", valid_pred, valid_true], ["train", train_pred, train_true]]:
        ref_metrics[('ref_model', 'f1score', k)].append(metrics.f1_score(true, pred))
        ref_cov = cov_score(true, pred)
        ref_metrics[('ref_model', 'coverage0', k)].append(ref_cov[0])
        ref_metrics[('ref_model', 'coverage1', k)].append(ref_cov[1])

In [10]:
df_ref_metrics = pd.DataFrame.from_dict(ref_metrics, orient='index')
mean_ref_metrics = df_ref_metrics.mean(axis=1)
f1_ref_prompts = [f"{'Ref_Model':<15}{mean_ref_metrics[('ref_model', 'f1score', 'valid')]:.3f}{sep_model}{'':<20}",
                  f"{'Ref_Model':<15}{mean_ref_metrics[('ref_model', 'f1score', 'train')]:.3f}{sep_model}{'':<20}"]
cov_ref_prompts = [f"{'Ref_Model':<15}{mean_ref_metrics[('ref_model', 'coverage0', 'valid')]:.3f}{sep_model}{'Ref_Model':<15}{mean_ref_metrics[('ref_model', 'coverage1', 'valid')]:.3f}",
                   f"{'Ref_Model':<15}{mean_ref_metrics[('ref_model', 'coverage0', 'train')]:.3f}{sep_model}{'Ref_Model':<15}{mean_ref_metrics[('ref_model', 'coverage1', 'train')]:.3f}"]
print(f"Average  :            {'Valid':^49}{sep_tv}{'Train':^49}",
      f"\tF1 score      {combine_prompts(f1_ref_prompts, sep_tv)}",
      f"\tCoverage      {combine_prompts(cov_ref_prompts, sep_tv)}",
      sep='\n')

Average  :                                  Valid                          ||                          Train                      
	F1 score      Ref_Model      0.755    |                            ||    Ref_Model      1.000    |                        
	Coverage      Ref_Model      0.791    |    Ref_Model      0.735    ||    Ref_Model      1.000    |    Ref_Model      1.000
